In [1]:
import sys
sys.path.append('./MatchZoo-py/')

In [2]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [3]:
preprocessor = mz.models.ArcI.get_default_preprocessor()

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6891.15it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3999.86it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2384687.39it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8085.60it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3708.65it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 639566.86it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 6479.33it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [5]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fdcb7578d30>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fdcb766ddd8>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [8]:
padding_callback = mz.models.ArcI.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=100,
    pad_word_value=0,
    pad_word_mode='pre',
    with_ngram=False
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [9]:
model = mz.models.ArcI()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['left_filters'] = [128]
model.params['left_kernel_sizes'] = [3]
model.params['left_pool_sizes'] = [4]
model.params['right_filters'] = [128]
model.params['right_kernel_sizes'] = [3]
model.params['right_pool_sizes'] = [4]
model.params['conv_activation_func'] = 'relu'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 100
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.9

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ArcI(
  (embedding): Embedding(30058, 300)
  (conv_left): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (conv_right): Sequential(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): ReLU()
      (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (dropout): Dropout(p=0.9, inplace=False)
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3456, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  9593859


In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-102 Loss-0.912]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5313 - normalized_discounted_cumulative_gain@5(0.0): 0.5878 - mean_average_precision(0.0): 0.5514



[Iter-204 Loss-0.730]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5746 - normalized_discounted_cumulative_gain@5(0.0): 0.6233 - mean_average_precision(0.0): 0.588



[Iter-306 Loss-0.643]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5834 - normalized_discounted_cumulative_gain@5(0.0): 0.6392 - mean_average_precision(0.0): 0.6048



[Iter-408 Loss-0.552]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5839 - normalized_discounted_cumulative_gain@5(0.0): 0.6298 - mean_average_precision(0.0): 0.5891



[Iter-510 Loss-0.466]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5811 - normalized_discounted_cumulative_gain@5(0.0): 0.6304 - mean_average_precision(0.0): 0.5894



[Iter-612 Loss-0.354]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5511 - normalized_discounted_cumulative_gain@5(0.0): 0.5986 - mean_average_precision(0.0): 0.5621



[Iter-714 Loss-0.300]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5404 - normalized_discounted_cumulative_gain@5(0.0): 0.5911 - mean_average_precision(0.0): 0.5511



[Iter-816 Loss-0.269]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5238 - normalized_discounted_cumulative_gain@5(0.0): 0.578 - mean_average_precision(0.0): 0.5416



[Iter-918 Loss-0.195]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5087 - normalized_discounted_cumulative_gain@5(0.0): 0.5811 - mean_average_precision(0.0): 0.5339



[Iter-1020 Loss-0.179]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5147 - normalized_discounted_cumulative_gain@5(0.0): 0.5696 - mean_average_precision(0.0): 0.5353

Cost time: 1298.3376731872559s
